In [1]:
!cp '/content/drive/MyDrive/Colab Notebooks/02-14-2018.csv' .

In [ ]:
!pip3 install datasets

In [96]:
import glob
import pandas as pd
import numpy as np
from itertools import combinations
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [97]:
csv_files = glob.glob("*.csv")

dataset = pd.DataFrame()
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dataset = pd.concat([dataset, df], axis = 0)

In [ ]:
dataset['Label'].value_counts()

In [99]:
dataset.columns=dataset.columns.str.strip()

zero_variance_cols=[]
for col in dataset.columns:
  if len(dataset[col].unique()) == 1:
    zero_variance_cols.append(col)

dataset.drop(columns=zero_variance_cols, axis=1, inplace=True)

dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

column_pairs = [(i, j) for i, j in combinations(dataset, 2) if dataset[i].equals(dataset[j])]
ide_cols = []
for col_pair in column_pairs:
    ide_cols.append(col_pair[1])
dataset.drop(columns=ide_cols, axis=1, inplace=True)

In [100]:
dataset.replace(to_replace=['FTP-BruteForce'], value=1, inplace=True)
dataset.replace(to_replace=['SSH-Bruteforce'], value=1, inplace=True)
dataset.replace(to_replace=['Benign'], value=0, inplace=True)

In [ ]:
dataset['Label'].value_counts()

In [102]:
df1 = dataset[dataset['Label'] == 0][:60]
df2 = dataset[dataset['Label'] == 1][:60]

In [103]:
df = pd.concat([df1, df2], axis=0)

In [104]:
df['text'] = df[df.columns[0:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

In [105]:
df = df[df.columns.intersection(['text', 'Label'])]
df = df.sample(frac=1)
df['label']=df['Label']
df.drop(columns='Label', axis=1, inplace=True)

In [106]:
df.to_json("openaiDataset.jsonl", orient='records', lines=True)

In [107]:
train, test = train_test_split(df, test_size=0.2)

In [108]:
train.to_json("hfTrainingDataset.jsonl", orient='records', lines=True)
test.to_json("hfTestingDataset.jsonl", orient='records', lines=True)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset = load_dataset('json', data_files={'train': 'hfTrainingDataset.jsonl', 'test': 'hfTestingDataset.jsonl'})
dataset.push_to_hub('chathuru/cicids2018')